# Analysis for case study 3

In [5]:
import pickle
import numpy as np
import pandas as pd

## Load in data

In [10]:
# load data
ds = ['carla']
data = []
for d in ds:      
    tab_file = 'study-3-collaborative-{}-seq-res-EXP2.p'.format(d)
    with open(tab_file, 'rb') as f:
        data.append(pickle.load(f))
        
# convert to dataframe
df = pd.concat(pd.DataFrame.from_dict(dat) for dat in data)
df.head()

,Case,Dataset,Trial,Metrics_perception_object_3d_tot_TP,Metrics_perception_object_3d_tot_FP,Metrics_perception_object_3d_tot_FN,Metrics_perception_object_3d_tot_T,Metrics_perception_object_3d_mean_precision,Metrics_perception_object_3d_mean_recall,Metrics_tracking_HOTA_HOTA,...,Metrics_tracking_Count_Frames,Metrics_prediction_agg_ADE,Metrics_prediction_agg_FDE,Metrics_prediction_std_ADE,Metrics_prediction_std_FDE,Metrics_prediction_n_with_truth,Metrics_prediction_n_objects,Metrics_collaborative_n_collab_detections_median,Metrics_collaborative_n_collab_sensors_in_view_median,Metrics_collaborative_n_collab_sensors_total_median
0,1,CARLA,0,405,0,155,560,1.0,0.728783,"[0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426, 0.8003924183032426]",...,50,4.058053,8.993276,0.912303,2.049654,1074,1074,54.0,5.0,39.0
1,2,CARLA,0,405,0,155,560,1.0,0.728783,"[0.7904860231182623, 0.7904860231182623, 0.7904860231182623, 0.7904860231182623, 0.7904860231182623, 0.7904860231182623, 0.7896817111668607, 0.7896817111668607, 0.7896817111668607, 0.7896817111668607, 0.7896817111668607, 0.7896817111668607, 0.7896817111668607, 0.7896817111668607, 0.7896817111668607, 0.7896817111668607, 0.7896817111668607, 0.7896817111668607, 0.7896817111668607]",...,50,3.986209,9.069929,0.834006,1.627570,1134,1134,54.0,5.0,39.0
2,3,CARLA,0,405,0,155,560,1.0,0.728783,"[0.7744893161360097, 0.7744893161360097, 0.7744893161360097, 0.7744893161360097, 0.7744893161360097, 0.7744893161360097, 0.7744893161360097, 0.7744893161360097, 0.7744893161360097, 0.7744893161360097, 0.7744893161360097, 0.7744893161360097, 0.7737673298695505, 0.7737673298695505, 0.7737673298695505, 0.7737673298695505, 0.7737673298695505, 0.7737673298695505, 0.7737673298695505]",...,50,4.096320,8.868105,0.928516,1.920644,1010,1010,24.5,5.0,39.0
3,4,CARLA,0,405,0,155,560,1.0,0.728783,"[0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132, 0.7363958967625132]",...,50,3.065523,5.872674,2.160766,4.997837,368,368,NaN,NaN,NaN
4,1,CARLA,1,296,0,256,552,1.0,0.546256,"[0.4636456581018891, 0.4636456581018891, 0.4636456581018891, 0.4636456581018891, 0.4636456581018891, 0.4636456581018891, 0.4636456581018891, 0.4636456581018891, 0.4636456581018891, 0.4636456581018891, 0.4636456581018891, 0.4636456581018891, 0.4636456581018891, 0.4636456581018891, 0.4626967520316524, 0.4626967520316524, 0.4626967520316524, 0.4626967520316524, 0.4626967520316524]",...,50,2.603682,6.122112,1.804317,4.663590,1320,1320,26.5,2.0,39.0


## Extract Data of Interest

Rows: case numbers

Columns: metrics

Within columns: each dataset's mean and std values across trials

In [11]:
metrics_of_interest = {
                            'Metrics_collaborative_n_collab_sensors_in_view_median':'Collab: \#S-in-range/frame',
                            'Metrics_collaborative_n_collab_detections_median':'Collab: Dets/frame',
                            'Metrics_tracking_HOTA_HOTA':'Trk: HOTA',
                            'Metrics_tracking_CLEAR_MOTA':'Trk: MOTA',
                            'Metrics_tracking_CLEAR_MOTP':'Trk: MOTP',
                            'Metrics_prediction_std_ADE':'Pred: ADE',
                            'Metrics_prediction_std_FDE':'Pred: FDE'}

expansion_types = {
                    'Metrics_collaborative_n_collab_sensors_in_view_median':None,
                    'Metrics_collaborative_n_collab_detections_median':None,
                    'Metrics_tracking_HOTA_HOTA':'value-at-middle',
                    'Metrics_tracking_CLEAR_MOTA':'value-at-middle',
                    'Metrics_tracking_CLEAR_MOTP':'value-at-middle',
                    'Metrics_prediction_std_ADE':None,
                    'Metrics_prediction_std_FDE':None}

In [12]:
# Convert to table format in triple slash format
mark_best_in_cell = False
mask_best_in_col = False

# subrow_formatter = '\tworowsubtablecenter{{{}}}{{\tworowsubtablecenter{{{}}}{{{}}}}}'
subrow_formatter = '{}'

dses = df["Dataset"].unique()
print(f'Triple slash dataset ordering is: {dses}')
res_triple_agg = []
for i_case in df['Case'].unique():
    res_triple = {'Case':i_case}
    res_triple.update({'Data':subrow_formatter.format(*[d[0].upper() for d in dses])})
    for met_k, met_v in metrics_of_interest.items():
        res_met_triple = []
        for dataset in df['Dataset'].unique():
            met_res = df[(df['Case'] == i_case) & (df['Dataset'] == dataset)][met_k]
            if expansion_types[met_k] == 'max-over-dict':
                met_vals = [np.max(met.values()) for met in met_res]
            elif expansion_types[met_k] == 'max-over-list':
                met_vals = [np.max(met) for met in met_res]
            elif expansion_types[met_k] == 'value-at-middle':
                met_vals = [np.median(met) for met in met_res]  # median accomplishes middle value
            elif expansion_types[met_k] is None:
                met_vals = met_res
            else:
                raise NotImplementedError('Expansion {} not implemented'.format(expansion_types[met_k]))
            mn, std = np.nanmedian(met_vals), np.nanstd(met_vals)
            res_met_triple.append((mn, std))
            
        # triple slash format -- underlining best in cell
        if mark_best_in_cell and (not all([np.isnan(mn) for mn, _ in res_met_triple])):
            best_idx = metric_best_evaluator[met_k]([mn for mn, _ in res_met_triple])
        else:
            best_idx = None
        res_met_triple_new = []
        for i, (mn, std) in enumerate(res_met_triple):
            if np.isnan(mn):
                wstr = 'N/A'
            else:
                if (best_idx is not None) and (i==best_idx):
                    wstr = f'\\underline{{{mn:4.2f} +/- {std:4.2f}}}'
                else:
                    wstr = f'{mn:4.2f} +/- {std:4.2f}'
            res_met_triple_new.append(wstr)

        # Format the whole triple
        res_trip = subrow_formatter.format(*res_met_triple_new)
        res_triple.update({met_v:res_trip})
    res_triple_agg.append(res_triple)

Triple slash dataset ordering is: ['CARLA']


/home/spencer/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/spencer/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [13]:
pd.set_option('display.max_colwidth',1000)
df_triple = pd.DataFrame(res_triple_agg)
lat_str = df_triple.to_latex(index=False, multirow=True, escape=False).replace('\\\\\n', '\\\\ \\midrule\n')
print(lat_str)

\begin{tabular}{rllllllll}
\toprule
 Case & Data & Collab: \#S-in-range/frame & Collab: Dets/frame &     Trk: HOTA &      Trk: MOTA &     Trk: MOTP &     Pred: ADE &     Pred: FDE \\ \midrule
\midrule
    1 &    C &              3.50 +/- 1.50 &    40.25 +/- 13.75 & 0.63 +/- 0.17 & -0.06 +/- 0.68 & 3.04 +/- 0.07 & 1.36 +/- 0.45 & 3.36 +/- 1.31 \\ \midrule
    2 &    C &              3.50 +/- 1.50 &    40.25 +/- 13.75 & 0.61 +/- 0.18 & -0.08 +/- 0.69 & 2.90 +/- 0.07 & 0.98 +/- 0.14 & 2.23 +/- 0.60 \\ \midrule
    3 &    C &              3.50 +/- 1.50 &     18.50 +/- 6.00 & 0.60 +/- 0.18 & -0.09 +/- 0.67 & 2.88 +/- 0.06 & 0.94 +/- 0.01 & 2.05 +/- 0.13 \\ \midrule
    4 &    C &                        N/A &                N/A & 0.66 +/- 0.08 &  0.54 +/- 0.07 & 3.02 +/- 0.10 & 1.60 +/- 0.56 & 5.74 +/- 0.74 \\ \midrule
\bottomrule
\end{tabular}



/tmp/ipykernel_1237927/1648327297.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  lat_str = df_triple.to_latex(index=False, multirow=True, escape=False).replace('\\\\\n', '\\\\ \\midrule\n')
